### Phase 1: Data Scraping, Extraction, Cleaning, and Exporting

In this phase, I scrape the Islamabad, Pakistan air quality data available on Pakistan Environmental Protection Agency website, extract the data from the .pdf files, clean it and export it to .csv files.

I divide the scraped .pdfs files by whether or not the files have tables of data or images of the tables of data. The .pdf files that have tables of data are further divided based on whether or not the tabula-py package reads them correctly or not. Each group of files is then separately treated, cleaned, and saved.

To read the data from the .pdf files containing images of data, the initial idea was to use pytesseract to read them. However, there were serious issues related to it's accuracy. For example, pytesseract was not accurate in identifying the presence of decimal points in the numbers in the data, leading to genuine data quality concerns. Therefore, I decided to utilize the Google Cloud Vision API which was really accurate as compared to pytesseract.

Each of the images of data and the tables extracted from these images are displayed together in this notebook towards the end for manual verification purposes as well. I also export a merged .csv file called "final_data.csv" that contains the observations of Islamabad air quality for every month from June, 2019 to March, 2023. A new column named "Year" is added in this file to easily distinguish the observations by year.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import fitz
import io
from PIL import Image
import copy
import re
import requests
from lxml import html
import camelot
import tabula
from PyPDF2 import PdfReader


In [ ]:
url = "https://environment.gov.pk/Detail/ZjU5NDM3YjItNTdiOS00NTk5LWExYzUtMjI2NzE5YjdlOGM5"

w_page = requests.get(url)

In [ ]:
links = html.fromstring(w_page.content)

links = links.xpath('//a/@href')

extract_ext = ".pdf"

In [ ]:
cleaned_links = []

for link in links:
    if extract_ext in link:
        if link not in cleaned_links:
            cleaned_links.append(link)
cleaned_links

In [ ]:
filter_string_list = ["2018", "May2019", "May23"] 

filtered_links = copy.deepcopy(cleaned_links)

for f in filter_string_list:
    for link in cleaned_links:
        if f in link:
            filtered_links.remove(link)

filtered_links

In [ ]:
epa = "https://environment.gov.pk"

In [ ]:
for l in filtered_links:
    
    file_name = re.search('([^\/]+)\.pdf$', l)
    
    file_name = file_name.group(0)
    
    file_url = epa + l
    
    
    d_file = requests.get(file_url)
    
    open('{}'.format(file_name), 'wb').write(d_file.content)
    

In [ ]:
files = os.listdir()

files = [file for file in files if ".pdf" in file]


tablefiles = []
imgfiles = []

for f in files:
    
    pdf = fitz.open(f)

    page = pdf[0]

    images = page.get_images(full=True)
    
    if images == []:
        tablefiles.append(f)
    else:
        imgfiles.append(f)


In [ ]:
output_dir = "extracted_images"
output_frmt = "png"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
image_names = []

for i in imgfiles:    
    pdf = fitz.open(i)
    name = i.replace(".pdf","")
    image_names.append(name)

    page = pdf[0]

    images = page.get_images(full=True)

    xref = images[0][0]
    
    base_image = pdf.extract_image(xref)
    image_bytes = base_image["image"]

    image = Image.open(io.BytesIO(image_bytes))
    image.save(open(os.path.join(output_dir, "{}.png".format(name)), "wb"), format=output_frmt.upper())


In [ ]:
for i in image_names:
    print(i)

In [ ]:
from img2table.document import Image

from img2table.ocr import VisionOCR

ocr = VisionOCR(api_key="", timeout=15) # Google Cloud Vision API key

for i in image_names:
    
    extracted_img = Image("extracted_images\\{}.png".format(i))
    # Table extraction
    extracted_img.to_xlsx(dest="{}.xlsx".format(i),
                ocr=ocr,
                implicit_rows=False,
                borderless_tables=False,
                min_confidence=50)

In [ ]:
tablefiles

In [ ]:
table_dfs_list = []

readable_files = []

incorrectly_read_files = []

somewhat_readable = []

for tf in tablefiles:
    
    year = re.search("[0-9]+", tf)
    year = year.group(0)
    
    df_list = tabula.read_pdf(tf, pages=1, pandas_options = {"header":None})
    table_df = df_list[0]
    
    if len(table_df.columns.values) < 6:
        incorrectly_read_files.append(tf)
    elif len(table_df.columns.values) > 6:
        somewhat_readable.append(tf)
    else:
        readable_files.append(tf)
        table_df['Year'] = year
        table_dfs_list.append(table_df)

In [ ]:
len(incorrectly_read_files)

In [ ]:
len(somewhat_readable)

In [ ]:
len(table_dfs_list)

In [ ]:
len(readable_files)

In [ ]:
for i in table_dfs_list:
    print(i)

In [ ]:
col_names = ["Date", "Temperature", "Humidity", "NO2", "SO2", "PM2.5", "Year"]

cleaned_cr = []

for df in table_dfs_list:
    if pd.isnull(df[0][0]):
        cleaned_df = df.drop(0)
        if df[0][1]=="Date":
            cleaned_df = cleaned_df.drop(1)
        if "NEQS" in df[0][2]:
            cleaned_df = cleaned_df.drop(2)
        ext_month = cleaned_df.iloc[7, 0]
        month = re.search("[A-Za-z]+", ext_month)
        month = month.group(0)
        
        while 1:
            if month not in cleaned_df.iloc[-1, 0]:
                cleaned_df = cleaned_df[:-1]
            else:
                break

        cleaned_df.reset_index(drop=True, inplace=True)
        cleaned_df.columns = col_names
        cleaned_cr.append(cleaned_df)
    else:
        if df[0][0] == "Date":
            cleaned_df = df.drop(0)
            if "NEQS" in df[0][1]:
                cleaned_df = cleaned_df.drop(1)
            if "Value" in df[0][2]:
                cleaned_df = cleaned_df.drop(2)
            ext_month = cleaned_df.iloc[5, 0]
            month = re.search("[A-Za-z]+", ext_month)
            month = month.group(0)
            cleaned_df = cleaned_df[:-1]
            while 1:
                if month not in cleaned_df.iloc[-1, 0]:
                    cleaned_df = cleaned_df[:-1]
                else:
                    break
            cleaned_df.reset_index(drop=True, inplace=True)
            cleaned_df.columns = col_names
            cleaned_cr.append(cleaned_df)
        else:
            cleaned_df = df
            cleaned_df.iloc[0,2] = cleaned_df[0][0]
            cleaned_df = cleaned_df.drop([0, 1])
            ext_month = cleaned_df.iloc[5, 0]
            month = re.search("[A-Za-z]+", ext_month)
            month = month.group(0)
            cleaned_df = cleaned_df[:-1]
            while 1:
                if month not in cleaned_df.iloc[-1, 0]:
                    cleaned_df = cleaned_df[:-1]
                else:
                    break
            cleaned_df.reset_index(drop=True, inplace=True)
            cleaned_df.columns = col_names
            cleaned_cr.append(cleaned_df)
    

In [ ]:
for c in cleaned_cr:
    print(c)

In [ ]:
for s in somewhat_readable:
    print(s)

In [ ]:
cleaned_sr = []

for s in somewhat_readable:
    year = re.search("[0-9]+", s)
    year = year.group(0)
    
    reader = PdfReader(s)
    page = reader.pages[0]
    string_t = page.extract_text()

    string_bytes = io.StringIO(string_t)

    df = pd.read_csv(string_bytes, sep = "\n")
    df = df[df.columns.values[0]].str.split(" ", expand=True)
    df = df.iloc[:, :6]
    
    df["Year"] = year
    
    ext_month = df.iloc[0, 0]
    month = re.search("[A-Za-z]", ext_month)
    month = month.group(0)
    
    while month not in df.iloc[-1, 0]:
        df = df[:-1]
    df.columns = col_names
    cleaned_sr.append(df)
    

In [ ]:
for s in cleaned_sr:
    print(s)
    

In [ ]:
all_clean_list = cleaned_cr + cleaned_sr

In [ ]:
len(cleaned_cr)

In [ ]:
len(cleaned_sr)

In [ ]:
for i in incorrectly_read_files:
    print(i)

In [ ]:
cleaned_ir = []

for i in incorrectly_read_files:
    year = re.search("[0-9]+", i)
    year = year.group(0)

    reader = PdfReader(i)
    page = reader.pages[0]
    string_t = page.extract_text()

    string_bytes = io.StringIO(string_t)

    df = pd.read_csv(string_bytes, sep = "\n")
    df = df.drop(0)
    df.reset_index(drop=True, inplace=True)
    df = df[:-1]
    
    ext_month = df.iloc[7, 0]
    month = re.search("[A-Za-z]", ext_month)
    month = month.group(0)
    
    
    while 1:
        if month not in df.iloc[0, 0]:
            df = df.drop(0)
            df.reset_index(drop=True, inplace=True)
        else:
            break
    df.columns = ["Date Temperature Humidity NO2 SO2 PM2.5"]
    df = df[df.columns.values[0]].str.split(" ", expand=True)
    if len(df.columns.values) == 7:
        df = df.drop(6, axis=1)
    if len(df.columns.values) > 7:
        df["hum"] = df[2] + df[3]
        df = df.drop(2, axis=1)
        df = df.drop(3, axis=1)
        df["no2"] = df[4] + df[5]
        df = df.drop(4, axis=1)
        df = df.drop(5, axis=1)
        df["so2"] = df[6] + df[7]
        df = df.drop(6, axis=1)
        df = df.drop(7, axis=1)
        df["pm"] = df[8] + df[9]
        df = df.drop(8, axis=1)
        df = df.drop(9, axis=1)
        df.replace("", float("NaN"), inplace=True)
        df.dropna(how='all', axis=1, inplace=True)

    df["Year"] = year
    df.reset_index(drop=True, inplace=True)
    df.columns = col_names
    cleaned_ir.append(df)


In [ ]:
for c in cleaned_ir:
    print(c)

In [ ]:
len(cleaned_ir)

In [ ]:
all_clean_list = all_clean_list + cleaned_ir
len(all_clean_list)

In [ ]:
xlsx_files = os.listdir()
xlsx_files = [f for f in xlsx_files if ".xlsx" in f]

xlsx_files

In [ ]:
cleaned_xlf = []

for j in xlsx_files:
    t = pd.read_excel(j, header=1)
    year = re.search("\d{4}\.\w+", j)
    year = year.group(0)
    year = year.replace(".xlsx", "")
    t = t.drop(0)
    t.reset_index(drop=True, inplace=True)
    while 1:
        if "NEQS" in t.loc[0][0]:
            t = t.drop(0)
            t.reset_index(drop=True, inplace=True)
        elif "Value" in t.loc[0][0]:
            t = t.drop(0)
            t.reset_index(drop=True, inplace=True)
        else:
            break
    t = t[:-1]
    t = t.dropna(axis='columns', how='all')
    t["Year"] = year
    t.columns = col_names
    t.reset_index(drop=True, inplace=True)
    cleaned_xlf.append(t)

In [ ]:
for x in cleaned_xlf:
    print(x)

In [ ]:
len(cleaned_xlf)

In [ ]:
len(cleaned_xlf) == len(xlsx_files)

In [ ]:
all_clean_list = all_clean_list + cleaned_xlf

In [ ]:
len(all_clean_list) == len(files)

In [ ]:
from IPython.display import Image, display

img_path = "extracted_images"

disp = os.listdir("extracted_images/")

for i in range(len(disp)):
    print("Image {}:".format(i+1))
    display(Image(img_path + "/" +disp[i], width=400))
    print("Extracted Dataframe {}:".format(i+1))
    print(cleaned_xlf[i])

In [ ]:
extracted_dir = "extracted_data"

if not os.path.exists(extracted_dir):
    os.makedirs(extracted_dir)

In [ ]:
for c in range(len(readable_files)):
    name = readable_files[c].replace(".pdf", "")
    df = cleaned_cr[c].drop("Year", axis=1)
    df.to_csv(os.path.join(extracted_dir, "{}.csv".format(name)), index=False)

for c in range(len(somewhat_readable)):
    name = somewhat_readable[c].replace(".pdf", "")
    df = cleaned_sr[c].drop("Year", axis=1)
    df.to_csv(os.path.join(extracted_dir, "{}.csv".format(name)), index=False)

for c in range(len(incorrectly_read_files)):
    name = incorrectly_read_files[c].replace(".pdf", "")
    df = cleaned_ir[c].drop("Year", axis=1)
    df.to_csv(os.path.join(extracted_dir, "{}.csv".format(name)), index=False)
    
for x in range(len(xlsx_files)):
    name = xlsx_files[x].replace(".xlsx", "")
    df = cleaned_xlf[x].drop("Year", axis=1)
    df.to_csv(os.path.join(extracted_dir, "{}.csv".format(name)), index=False)

In [ ]:
data_files = os.listdir("extracted_data")

In [ ]:
len(data_files)

In [ ]:
len(files)

In [ ]:
len(data_files) == len(files)

In [ ]:
len(all_clean_list)

In [ ]:
final_df = pd.concat(all_clean_list)

In [ ]:
final_df["Date"] = final_df["Date"].str.replace('\s+', '', regex=True)
final_df["Date"] = final_df["Date"].str.replace('-', ' ', regex=True)
final_df["Date"] = final_df["Date"].str.replace('\d{4}', '', regex=True)
final_df["Date"] = final_df["Date"].str.replace('^0+(?!$)', '', regex=True)
final_df["Date"] = final_df["Date"].str.replace('\d{2}$', '', regex=True)
final_df["Date"] = final_df["Date"].str.rstrip()


final_df

In [ ]:
len(final_df["Date"].unique())

In [ ]:
final_df.to_csv("final_data.csv", index=False)